In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
class PCA():
   
    # ---------------------------------------------------------------------------
    # Método construtor
    # Função que será chamada toda vez que um objeto PCA for inicializado
    # O modelo selecionará quantos PCs forem necessários para cumprir o % de variabilidade explicada especificado em a

    def __init__ (self, a=0.9):

        # se 0<=a<1,  'a' indica a fraçao de variancia explicada desejada
        # se a>=1,    'a' indica o numero de componentes desejado
        self.a = a
   
    # ---------------------------------------------------------------------------
    # Função para treino do modelo
    # X são os dados de treino
    
    def fit(self, X, conf_Q=0.99, conf_T2=0.99, plot=True):
    
        # guardando médias e desvios-padrão do treino (de cada coluna)
        self.mu_train = X.mean(axis=0)
        self.std_train = X.std(axis=0)        
       
        # normalizando dados de treino
        X = np.array(((X - self.mu_train)/self.std_train))
       
        # calculando a matriz de covariâncias dos dados
        Cx = np.cov(X, rowvar=False)
        
        # aplicando decomposição em autovalores e autovetores
        self.L, self.P = np.linalg.eig(Cx)
        
        # frações da variância explicada
        fv = self.L/np.sum(self.L)
        
        # frações da variância explicada acumuladas
        fva = np.cumsum(self.L)/sum(self.L)
       
        # definindo número de componentes
        if (self.a>0 and self.a<1):
            self.a = np.where(fva>self.a)[0][0]+1 
            
        # calculando limites de detecção

        # limite da estatística T^2
        from scipy.stats import f
        F = f.ppf(conf_T2, self.a, X.shape[0]-self.a)
        self.T2_lim = ((self.a*(X.shape[0]**2-1))/(X.shape[0]*(X.shape[0]-self.a)))*F
        
        # limite da estatística Q
        theta = [np.sum(self.L[self.a:]**(i)) for i in (1,2,3)]
        ho = 1-((2*theta[0]*theta[2])/(3*(theta[1]**2)))
        from scipy.stats import norm
        nalpha = norm.ppf(conf_Q)
        self.Q_lim = (theta[0]*(((nalpha*np.sqrt(2*theta[1]*ho**2))/theta[0])+1+
                                ((theta[1]*ho*(ho-1))/theta[0]**2))**(1/ho))
        
        # plotando variâncias explicadas
        if plot:
            fig, ax = plt.subplots()
            ax.bar(np.arange(len(fv)),fv)
            ax.plot(np.arange(len(fv)),fva)
            ax.set_xlabel('Número de componentes')
            ax.set_ylabel('Variância dos dados')
            ax.set_title('PCA - Variância Explicada');

    # ---------------------------------------------------------------------------
    # Função para teste do modelo
            
    def predict(self, X):
            
        # normalizando dados de teste (usando os parâmetros do treino!)
        X = np.array((X - self.mu_train)/self.std_train)

        # calculando estatística T^2
        T = X@self.P[:,:self.a]
        self.T2 = np.array([T[i,:]@np.linalg.inv(np.diag(self.L[:self.a]))@T[i,:].T for i in range(X.shape[0])])

        # calculando estatística Q
        e = X - X@self.P[:,:self.a]@self.P[:,:self.a].T
        self.Q  = np.array([e[i,:]@e[i,:].T for i in range(X.shape[0])])
        
        # calculando contribuições para Q
        self.c = np.absolute(X*e) 
                
    # ---------------------------------------------------------------------------
    # Função para plotar cartas de controle
    
    def plot_control_charts(self, fault = None):
     
        fig, ax = plt.subplots(1,2, figsize=(15,3))

        ax[0].semilogy(self.T2,'.')
        ax[0].axhline(self.T2_lim,ls='--',c='r');
        ax[0].set_title('Carta de Controle $T^2$')
        
        ax[1].semilogy(self.Q,'.')
        ax[1].axhline(self.Q_lim,ls='--',c='r')
        ax[1].set_title('Carta de Controle Q')
 
        if fault is not None:
            ax[0].axvline(fault, c='w')
            ax[1].axvline(fault, c='w')

    # ---------------------------------------------------------------------------
    # Função para plotar mapas de contribuição
            
    def plot_contributions(self, fault=None, index=None, columns=None):

        fig, ax = plt.subplots(figsize=(20, 6))
        
        c = pd.DataFrame(self.c, index=index, columns=columns)
    
        sns.heatmap(c, ax=ax, yticklabels=int(self.c.shape[0]/10), cmap = plt.cm.Blues)
        
        ax.set_title('Contribuições parciais para Q')
        
        if fault is not None:
            ax.axhline(y=c.index[fault], ls='--', c='k')

In [8]:
# Aceita um DataFrame e retorna outro contendo variáveis atrasadas

def apply_lag(df, lag=1):
    
    # Aplica a função lagmat do statsmodels
    # 'trim' é o método de corte a ser usado: ‘forward’ corta as observações inválidas na frente
    # 'original' é como o array original é tratado: 'in' retorna o array original e o atrasado como dois arrays separados
    # [lag:,:] desconsidera as linhas nas quais observações foram zeradas - ocorre com a aplicação dos lags
    array_lagged = lagmat(df, maxlag=lag, trim="forward", original='in')[lag:,:]  
    new_columns = []
    
    # Armazena em um array os nomes das novas colunas atrasadas de acordo com quantos lags foram especificados
    for l in range(lag):
        new_columns.append(df.columns+'_lag'+str(l+1))
    
    # Unifica num array os nomes de todas as colunas, originais e atrasadas
    columns_lagged = df.columns.append(new_columns)
    
    # Define os indexes do novo df que será criado
    index_lagged = df.index[lag:]
    
    # Organiza o novo df, com o novo header, novos indexes e os dados originais e atrasados
    df_lagged = pd.DataFrame(array_lagged, index=index_lagged, columns=columns_lagged)
       
    return df_lagged 

In [9]:
def mspc_pca(data, IDV):
    
    df_test = data.query("STATUS == " + str(IDV)).copy()
    df_test.drop(['Time', 'Criticidade', 'HORIMETRO', 'STATUS', 'Run', 'MA(Run)'], 1, inplace=True)

    try:
        pca.predict(df_test)

        pca.plot_control_charts()
        plt.suptitle(f'ID({IDV})');

        pca.plot_contributions(columns=df_test.columns)

        print(f'Taxas de detecção de falhas - ID({IDV})')

        print(f'\nT2: {(pca.T2>pca.T2_lim).sum()/pca.T2.shape[0]}')
        print(f'Q: {(pca.Q>pca.Q_lim).sum()/pca.Q.shape[0]}')
    
    except:
        # pra quando o df fica vazio, por exemplo (o período não existe após algum drop)
        pass